# Data Ingest of 10-h Fuel Moisture Content

This notebook demonstrates retrieval and filtering of 10-h dead FMC data from RAWS. Retrieval of 10-h FMC observations is done with the software package `SynopticPy` and a stash of RAWS data kept and maintained by the broader OpenWFM community. This notebook will demonstrate use of `Synopticpy` with a free token, so limits are placed on the number of sensor hours that can be requested. Only records within the past year are freely available.

The main steps in the retrieval are:
* Use `synoptic.Metadata` to determine the RAWS with FMC data in the given spatial domain and time frame
* Use `synoptic.Timeseries` to retrieve all available data that may be relevant to FMC modeling. *NOTE:* the stations are selected so they must have FMC data, and then any other available variables are collected as a bonus. These data are used for exploratory purposes and quality control checks, but predictors for final modeling comes from HRRR.

For more info on python library API, see Brian Blaylock's `SynopticPy` [python package](https://github.com/blaylockbk/SynopticPy)

For more info on available Synoptic RAWS variables, see [Synoptic Data](https://demos.synopticdata.com/variables/index.html) documentation

## Setup

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import synoptic
import json
import sys
import polars as pl
sys.path.append('../src')
from utils import Dict

A configuration file is used to control data ingest. Automated processes utilize the file `training_data_config.json` or `forecast_config.json`. In this tutorial, we will manually build a config file

In [ ]:
end = datetime.now()

# Get 1 year previous date while handling leap year edge cases
try:
    start = end.replace(year=end.year - 1)
except ValueError:
    # Handle February 29th case (leap year)
    start = end.replace(month=2, day=28, year=end.year - 1)

print(f"Start Date of retrieval: {start}")
print(f"End Date of retrieval: {end}")

In [ ]:
config = Dict({
    'start_time': start, # String as YYYY-MM-DD_HH:mm:ss OR datetime object
    'end_time': end,
    'bbox': [40, -105, 45, -100], # [min_lat, min_lon, max_lat, max_lon]
    'raws_vars': ["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation", "pressure", "soil_moisture", "soil_temp", "snow_depth", "snow_accum", "wind_direction"]
})

config

## Stations MetaData

*Note*: the bounding box format used in `wrfxpy` is `[min_lat, min_lon, max_lat, max_lon]`. But, the bounding box format used by Synoptic is `[min_lon, min_lat, max_lon, max_lat]`.

In [ ]:
bbox = config.bbox
bbox_reordered = [bbox[1], bbox[0], bbox[3], bbox[2]]
raws_vars = config.raws_vars

In [ ]:
sts = synoptic.Metadata(
    bbox=bbox_reordered,
    vars=["fuel_moisture"], # Note we only want to include stations with FMC. Other "raws_vars" are bonus
    obrange=(start, end),
).df()

In [ ]:
sts

## Station Time Series

We loop over the station IDs found in the previous step and retrieve all available data and then format and clean.

*NOTE*: this process is not parallelized, as the same IP address is used for each request and parallization may result in issues

In [ ]:
# name_mapping = {
#     "air_temp":"temp", 
#     "fuel_moisture":"fm", 
#     "relative_humidity":"rh", 
#     "precip_accum":"rain",
#     "solar_radiation":"solar", 
#     "wind_speed":"wind", 
#     "precip_accum":"precip_accum", 
#     "soil_moisture":"soil_moisture"
# }

In [ ]:
def format_raws(df,
                static_cols = ["stid", "latitude", "longitude", "elevation", "name", "state", "id"], 
                weather_vars = ["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation", "soil_moisture"]):
    # Given input dataframe (the output of synoptic.TimeSeries), return formatted dictionary
    # Inputs:
    # df: (dataframe)
    # Returns: tuple of data, units   

    assert "fuel_moisture" in df["variable"], "fuel_moisture not detected in input dictionary"
    units = {} # stores units for variables
    
    
    for var in weather_vars:
        if var in df['variable']:
            df_temp = df.filter(df['variable'] == var)
            unit = df_temp['units'].unique()
            if len(unit) != 1:
                raise ValueError(f"Variable {var} has multiple values for units")
            units[var] = unit[0]
    
    dat = df.filter(pl.col("variable").is_in(weather_vars))
    dat = dat.pivot(
        values="value",
        index=["date_time"]+static_cols,
        on="variable"
    )

    print(f"Found {dat.shape[0]} FMC records")
    
    # Fix column units
    if "air_temp" in dat.columns and units['air_temp'] == "Celsius":
        print("Converting RAWS air temp from C to K")
        units['air_temp'] = "Kelvin"
        dat = dat.with_columns(
                (pl.col("air_temp")+273.15).alias("air_temp")
            )
        
    if 'elevation' in static_cols: # convert ft to meters
        print("Converting RAWS elevation from ft to meters")
        # loc['elevation'] = loc['elevation'] * 0.3048
        dat = dat.with_columns(
                (pl.col("elevation") * 0.3048).alias("elevation")
            )
        units['elevation'] = "m"    
        
        
    return dat, units

In [ ]:
df_temp = synoptic.TimeSeries(
        stid="CPTC2",
        start=start,
        end=end,
        vars=config.raws_vars
    ).df()

df_temp

In [ ]:
dat, units = format_raws(df_temp, weather_vars = config.raws_vars)

In [ ]:
units

In [ ]:
dat

In [ ]:
def get_static(df, static_cols = ["stid", "latitude", "longitude", "elevation", "name", "state", "id"]):
    """
    Given dataframe of timeseries observations from RAWS station, get dictionary of static info, such as identifiers and physical attributes of station.
    
    Args:
        df: Input dataframe with timeseries observations.
        static_cols: List of column names to extract static information from.
    
    Returns:
        A dictionary called "loc" containing the unique value for each column in static_cols.
    
    Raises:
        ValueError: If any column in static_cols has more than one unique value in the dataframe.
    """
    
    loc = {}
    for col in static_cols:
        if col in df.columns:
            unique_values = df[col].unique()
            if len(unique_values) == 1:
                loc[col] = unique_values[0]
            else:
                raise ValueError(f"Column '{col}' has more than one unique value: {unique_values}")
        else:
            raise KeyError(f"Column '{col}' not found in the dataframe.")
    return loc

In [ ]:
print(f"Attempting retrieval of RAWS from {start} to {end} within {bbox}")
print("~"*75)

raws_dict = {}

for st in sts['stid']:
    print("~"*50)
    print(f"Attempting retrival of station {st}")
    df = synoptic.TimeSeries(
        stid="CPTC2",
        start=start,
        end=end,
        vars=config.raws_vars
    ).df()
    
    dat, units = format_raws(df, weather_vars = config.raws_vars)
    loc = get_static(dat)
    raws_dict[st] = {
        'RAWS': dat,
        'units': units,
        'loc': loc,
        'misc': "Data retrieved using `synoptic.TimeSeries` and formatted with custom functions within `ml_fmda` project"
    }

In [ ]:
raws_dict.keys()

In [ ]:
raws_dict["BRLW4"].keys()

In [ ]:
raws_dict["BRLW4"]["units"]

In [ ]:
raws_dict["BRLW4"]["RAWS"]

In [ ]:
raws_dict["BRLW4"]["loc"]

In [ ]:
get_static(raws_dict["BRLW4"]["RAWS"])